In [1]:
# Importing libraries
import pandas as pd
import numpy

In [2]:
# Read csv file into a pandas dataframe

# possible missing values which could be found in Data
missing_values = ["n/a", "na", "--"]

# sep = sepreation between different values in data file
# in CSV(comma sepreated values) files, values are sepreated by ','
# loading CSV file & converting all missing values to NAN
df = pd.read_csv("../DataSet/Training.csv", na_values = missing_values, sep = ",")      # converting all empty as NAN

In [3]:
# dropping all the rows which contain any NAN values
df = df.dropna()

In [4]:
## replacing all missing values
## replacing all NAN values with 0 here
#df.fillna(0)

In [5]:
# At this point data is in logical form
# to being able to predict it should be in counting formate
# Main goal is to count all the different values which could be required
# Required Processed Data Arrangement is
#
# {
#     statusValueOne:{
#                 counts for only row containing statusValueOne
#                 symptom1: symptom1 count
#                 symptom2: symptom2 count
#                 .
#                 .
#                 .
#                 symptomN: symptomN count
#                 }
#
#     statusValueTwo:{
#                 counts for only row containing statusValueOne
#                 symptom1: symptom1 count
#                 symptom2: symptom2 count
#                 .
#                 .
#                 .
#                 symptomN: symptomN count
#                 }
# }
# To achieve this data is to be divide 1st into respective groups
# First division would be on the bases of diseses

In [6]:
# finding the diamentions of the data fram
(rows, columns) = df.shape
# rows = number of rows in data fram
# columns = number of columns in data fram

In [7]:
# let say data is arranged such a way
# all columns are different symptoms(causes) and status(effects)
# it is required to sepreate them

In [8]:
# sepreating symptoms and status
#
# statusColumns = all the columns which contain disease status(effects)
# symptomsColumns = all the columns which contain all the symptoms(causes)
#
# SYNTEX:
# statusColumns = list(range(from, upTo))
# symptomsColumns = list(range(from, upTo))
#
# NOTE: upTo column will also be included

# getting all the columns to be selected
#statusColumns = list(range(133, 133))
statusColumns = [132]
symptomsColumns = list(range(0, 132))

# sepreating data fram into status & symptoms
status = df.iloc[:, statusColumns]
symptoms = df.iloc[:, symptomsColumns]

In [9]:
# getting all the unique value in status columns
# so that could be divided (1st division)
# status[status.columns[0]].unique() = list of all the unique values in status columns (disease)
# all the different disease in given data
diseaseList = status[status.columns[0]].unique()

# arranged data structure to store all the counts
processedData = {}

# sepreating data on the bases of uniqueStatusValues & arranging
#for eachStatusColumn in statusColumns:
#    # for every selected status column
#    for eachUniqueValue in uniqueStatusValues:
#        processedData[eachUniqueValue] = df[status.columns[eachStatusColumn]].count(eachUniqueValue)

# df.columns[statusColumns][0] = exect column header to select
# df[df[df.columns[statusColumns][0]] == 'Fungal infection'] to extract all the rows which have 'Fungal infection'
# in it
for eachDisease in diseaseList:
    processedData[eachDisease] = df[df[df.columns[statusColumns][0]] == eachDisease]

# After this processing processedData would be like
#processedData = {
#    disease1 = {
#        all symptoms
#    }
#    disease2 = {
#        all symptoms
#    }
# }

In [10]:
# a data structure to store all the necessary count which could be used
countedData = dict()

#countedData = {
#     "disease0" : {
#         "Symptom0": {
#             "status0":count,
#             "status1":count,
#             .
#             .
#             .
#             "statusN":count,
#         }
#         "Symptom1": {
#             "status0":count,
#             "status1":count,
#             .
#             .
#             .
#             "statusN":count,
#         }
#         .
#         .
#         .
#         "SymptomN": {
#             "status0":count,
#             "status1":count,
#             .
#             .
#             .
#             "statusN":count,
#         }
#     }
#     .
#     .
#     .
#     "diseaseN" : {
#         "Symptom0": {
#             "status0":count,
#             "status1":count,
#             .
#             .
#             .
#             "statusN":count,
#         }
#         "Symptom1": {
#             "status0":count,
#             "status1":count,
#             .
#             .
#             .
#             "statusN":count,
#         }
#         .
#         .
#         .
#         "SymptomN": {
#             "status0":count,
#             "status1":count,
#             .
#             .
#             .
#             "statusN":count,
#         }
#     }
# }

# now we have to count the data respectivly
for eachDisease, symptomsList in processedData.items(): # traversing over all Disease
    
    # for all symptoms of select Disease
    for eachSymptom in symptomsList:
        # getting unique value from selected symptoms column (eachSymptom) 
        uniqueValuesInSymptomList = processedData[eachDisease][eachSymptom].unique()
        # uniqueValuesInSymptomList is a list containing all unique value
        
        # for all Unique values of selected symptoms
        for eachUniqueValue in uniqueValuesInSymptomList:
            
            # checking weather disease(key) exist or not
            # if not add one & inilize with empty dict
            if eachDisease not in countedData.keys():
                countedData[eachDisease] = {}
            # checking weather Symptom(sub-key) exist or not
            # if not add one & inilize with empty dict
            if eachSymptom not in countedData[eachDisease].keys():
                countedData[eachDisease].update({eachSymptom:{}})
            # checking weather unique value(sub-sub-key) exist or not
            # if not add one & inilize with 0 (as initial count)
            if eachUniqueValue not in countedData[eachDisease][eachSymptom].keys():
                countedData[eachDisease][eachSymptom].update({eachUniqueValue:0})
            
            # counting and updating the count in countedData
            countedData[eachDisease][eachSymptom][eachUniqueValue] = list(processedData[eachDisease][eachSymptom]).count(eachUniqueValue)
            

In [11]:
# Now, initial probabilities have to be calculated
# for a single Disease
# P(Disease) = (number of entries of that Disease)/(total entires in th data)

# we have a list of all the disease in "diseaseList"

# to store all the initial probabilties
diseaseProbabilityList = {}
# diseaseProbabilityList = {
#              disease0: probability,
#              disease1: probability,
#              .
#              .
#              .
#              diseaseN: probability,
# }

# len(status) = total entires in th data
# total entires in data set
totalEnteries = len(status)

# traversing for all listed diseases in "diseaseList" & calculating probabilities 
for eachDiseases in diseaseList:
    # list(status)[0] gives the header value which is passed to the data frame to select that column
    # list(status[list(status)[0]]).count(eachDiseases) = number of entries of that Disease(eachDiseases)
    # len(status) = total entires in th data
    
    # calculating probability
    probability = list(status[list(status)[0]]).count(eachDiseases)/totalEnteries
    
    # storing it
    diseaseProbabilityList[eachDiseases] = probability

In [12]:
def getCount(disease, symptom, status):
    # this functions returns the count by searching counted data
    if status in countedData[disease][symptom].keys():
        return countedData[disease][symptom][status]
    else:
        # minimum number possible
        return 0.001

In [13]:
# all the symptoms which are found in patient
#givenSymptomsList = [ "red_sore_around_nose"]

def runTests(givenSymptomsList):
    # an empty dict which will temperarly store all the probabilities
    # by which it could be predicted which is th most likly  disease
    probabilityResultsList = {}

    for eachDisease in diseaseList:
        # inilizing to avoid any error in calculation
        probability = 1
        # calculting probabily of having a disease based in symptoms
        for eachGivenSymptom in givenSymptomsList:
            probability = probability*(getCount(eachDisease, eachGivenSymptom, 1)/len(status[status[list(status)[0]] == eachDisease]))
    
            # multiplying the probability of havng disease P(diease)
            probability = probability*diseaseProbabilityList[eachDisease]
    
            # storing the data
            probabilityResultsList[eachDisease] = probability
            
    # finding sum of all the calculated probability
    prob_sum = 0
    for eachDisease, diseaseProbability in probabilityResultsList.items():
        prob_sum = prob_sum + diseaseProbability
    
    # dividing all calculated probabilies to there sum for normalization
    for eachDisease, diseaseProbability in probabilityResultsList.items():
        probabilityResultsList[eachDisease] = diseaseProbability/prob_sum
        
            
    
    # returning the calculated probabilties
    return probabilityResultsList

In [14]:
def predict(probabilityResultsList):
    # making decision what is the most probabily the disease
    disease = str()
    lastProbability = int()
    
    for eachDisease, diseaseProbability in probabilityResultsList.items():
        print(f"For disease {eachDisease} calculated probability is: {diseaseProbability}")
        if lastProbability < diseaseProbability:
            lastProbability = diseaseProbability
            disease = eachDisease
    return disease

In [16]:
predict(runTests(["itching", "skin_rash","nodal_skin_eruptions", "dischromic _patches"]))

For disease Fungal infection calculated probability is: 0.9999999998139731
For disease Allergy calculated probability is: 7.35029852659718e-21
For disease GERD calculated probability is: 7.35029852659718e-21
For disease Chronic cholestasis calculated probability is: 8.379340320320784e-16
For disease Drug Reaction calculated probability is: 9.049687545946449e-11
For disease Peptic ulcer diseae calculated probability is: 7.35029852659718e-21
For disease AIDS calculated probability is: 7.35029852659718e-21
For disease Diabetes  calculated probability is: 7.35029852659718e-21
For disease Gastroenteritis calculated probability is: 7.35029852659718e-21
For disease Bronchial Asthma calculated probability is: 7.35029852659718e-21
For disease Hypertension  calculated probability is: 7.35029852659718e-21
For disease Migraine calculated probability is: 7.35029852659718e-21
For disease Cervical spondylosis calculated probability is: 7.35029852659718e-21
For disease Paralysis (brain hemorrhage) cal

'Fungal infection'